In [ ]:
#imports 
import pandas as pd
from pathlib import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.ensemble import IsolationForest
import numpy as np
from sklearn.neighbors import LocalOutlierFactor
import matplotlib.pyplot as plt
from estimators import Estimator
from estimators import Evaluator 

In [6]:
# Set the file paths for changing orders
co_path_1 = Path("../Data/Change-Orders-Export-2020-February-08-1609.csv")
co_path_2 = Path("../Data/Change-Orders-Export-2020-February-08-1618.csv")
co_path_3 = Path("../Data/Change-Orders-Export-2020-February-08-1632.csv")
co_path_4 = Path("../Data/Change-Orders-Export-2020-February-08-1633.csv")

#Set the file paths for tickets
tickets_path_1 = Path("../Data/Tickets-Export-2020-February-08-1621.csv")
tickets_path_2 = Path("../Data/Tickets-Export-2020-February-08-1623.csv")
tickets_path_3 = Path("../Data/Tickets-Export-2020-February-08-1639.csv")
tickets_path_4 = Path("../Data/Tickets-Export-2020-February-08-1640.csv")
tickets_path_5 = Path("../Data/Tickets-Export-2020-February-08-1644.csv")
tickets_path_6 = Path("../Data/Tickets-Export-2020-February-08-1647.csv")


# Read the CSVs for Changing orders
co_1_df = pd.read_csv(co_path_1)
co_2_df = pd.read_csv(co_path_2)
co_3_df = pd.read_csv(co_path_3)
co_4_df = pd.read_csv(co_path_4)

#Read the CSVs for tickets
ticket_1_df = pd.read_csv(tickets_path_1, header=0, escapechar='\\')
ticket_2_df = pd.read_csv(tickets_path_2, header=0, escapechar='\\')
ticket_3_df = pd.read_csv(tickets_path_3, header=0, escapechar='\\')
ticket_4_df = pd.read_csv(tickets_path_4, header=0, escapechar='\\')
ticket_5_df = pd.read_csv(tickets_path_5, header=0, escapechar='\\')
ticket_6_df = pd.read_csv(tickets_path_6, header=0, escapechar='\\')

#Concatenating changing orders and tickets dataframe
co_df = pd.concat([co_1_df, co_2_df, co_3_df, co_4_df], axis="rows", join="inner")
ticket_df = pd.concat([ticket_1_df, ticket_2_df, ticket_3_df, ticket_4_df, ticket_5_df, ticket_6_df], axis="rows", join='inner')


In [7]:
#Dropping Columns
co_df = co_df.drop(columns=['Content', 'Excerpt', 'Equipment Breakdown_type', 'Equipment Breakdown_quantity', 'Equipment Breakdown_unit', 'Equipment Breakdown_rate', 
                           'Material Breakdown_type', 'Material Breakdown_quantity', 'Material Breakdown_unit', 'Material Breakdown_rate', 'Author Username', 'Author Last Name',
                           'Author First Name', 'Parent', 'Parent Slug', 'Ping Status', 'Author Email', 'URL.1', 'URL', 'Title.1', 'Description', 'Alt Text', 'Caption', 'Featured',
                           'breakdowns_labor_0_type', 'breakdowns_labor_0_quantity', 'breakdowns_labor_0_hours', 'breakdowns_labor_0_rate', 'breakdowns_labor_0_type', 'breakdowns_material_0_type',
                           'breakdowns_material_0_quantity', 'breakdowns_material_0_unit', 'breakdowns_material_0_rate', 'breakdowns_equipment_0_type', 'breakdowns_equipment_0_quantity',
                           'breakdowns_equipment_0_unit', 'breakdowns_equipment_0_rate', '_validate_email', '__validate_email', 'Format', 'Template', 'Upload RFP', 'Exclusion', 'Add Tickets',
                           'Labor Breakdown_type', 'Labor Breakdown_quantity', 'Labor Breakdown_hours', 'Labor Breakdown_rate'], axis=1)

In [8]:
# Dropping Columns continued 
ticket_df = ticket_df.drop(columns=['Content', 'Excerpt', 'Equipment Breakdown_type', 'Equipment Breakdown_quantity', 'Equipment Breakdown_unit', 'Equipment Breakdown_rate', 
                           'Material Breakdown_type', 'Material Breakdown_quantity', 'Material Breakdown_unit', 'Material Breakdown_rate', 'Author Username', 'Author Last Name',
                           'Author First Name', 'Parent', 'Parent Slug', 'Ping Status', 'Author Email', 'URL.1', 'URL', 'Title.1', 'Description', 'Alt Text', 'Caption', 'Featured',
                           'breakdowns_labor_0_type', 'breakdowns_labor_0_quantity', 'breakdowns_labor_0_hours', 'breakdowns_labor_0_rate', 'breakdowns_labor_0_type', 'breakdowns_material_0_type',
                           'breakdowns_material_0_quantity', 'breakdowns_material_0_unit', 'breakdowns_material_0_rate', 'breakdowns_equipment_0_type', 'breakdowns_equipment_0_quantity',
                           'breakdowns_equipment_0_unit', 'breakdowns_equipment_0_rate', '_validate_email', '__validate_email', 'Format', 'Template',
                           'Labor Breakdown_type', 'Labor Breakdown_quantity', 'Labor Breakdown_hours', 'Labor Breakdown_rate', 'files_0_file', '_files_0_file', '_breakdowns_labor_0_rate',
                            '_breakdowns_material_0_rate', '_breakdowns_equipment_0_rate', 'Add Photos_file', 'Signed?', 'Labor Breakdown_rate_type', 'Signed?.1', 'Signed?.2', 'history'], axis=1)

In [9]:
#read in new data 
filepath = Path("../Data/TracfloDataFullSet.csv")
changeorder = pd.read_csv(filepath)
changeorder.head()

total  paid_total   balance  breakdowns_labor_0_hours  \
0  133000.0         0.0  133000.0                       8.0   
1   27248.0         0.0   27248.0                       8.0   
2   -5000.0         0.0   -5000.0                       0.0   
3   13852.0         0.0   13852.0                       0.0   
4       0.0         0.0       0.0                       8.0   

   breakdowns_labor_0_rate  breakdowns_material_0_rate   Slug     Total  \
0                      0.0                         0.0  193.0  133000.0   
1                      0.0                         0.0  248.0   27248.0   
2                      0.0                         0.0  251.0       0.0   
3                      0.0                         0.0  252.0       0.0   
4                      0.0                         0.0  253.0       0.0   

   Labor0Breakdown_quantity0total0  Labor0Breakdown_hourstotal  \
0                              0.0                         8.0   
1                              0.0                         8.0   
2                              0.0                         0.0   
3                              0.0                         0.0   
4                              0.0                         8.0   

   Labor0Breakdown_ratetotal  
0                      165.0  
1                        0.0  
2                        0.0  
3                        0.0  
4                        0.0

In [10]:
changeordermean=changeorder["breakdowns_labor_0_rate"]
changeordermean=changeordermean.replace(0,np.nan)
changeordermean=changeordermean.dropna()
changeordermean.head()
changeordermean.mean()


620.0230476190476

In [11]:
# marks 1 for data that is within labor rate of 1 to 800
changeorder["target"]=np.where(changeorder["breakdowns_labor_0_rate"].between(1,800),1,0)

In [12]:
changeorder.head()

total  paid_total   balance  breakdowns_labor_0_hours  \
0  133000.0         0.0  133000.0                       8.0   
1   27248.0         0.0   27248.0                       8.0   
2   -5000.0         0.0   -5000.0                       0.0   
3   13852.0         0.0   13852.0                       0.0   
4       0.0         0.0       0.0                       8.0   

   breakdowns_labor_0_rate  breakdowns_material_0_rate   Slug     Total  \
0                      0.0                         0.0  193.0  133000.0   
1                      0.0                         0.0  248.0   27248.0   
2                      0.0                         0.0  251.0       0.0   
3                      0.0                         0.0  252.0       0.0   
4                      0.0                         0.0  253.0       0.0   

   Labor0Breakdown_quantity0total0  Labor0Breakdown_hourstotal  \
0                              0.0                         8.0   
1                              0.0                         8.0   
2                              0.0                         0.0   
3                              0.0                         0.0   
4                              0.0                         8.0   

   Labor0Breakdown_ratetotal  target  
0                      165.0       0  
1                        0.0       0  
2                        0.0       0  
3                        0.0       0  
4                        0.0       0

In [13]:
#identify values outside of the mean 
potential_fraud=changeorder .loc[changeorder['target']==0]

In [14]:
potential_fraud.head()

total  paid_total   balance  breakdowns_labor_0_hours  \
0  133000.0         0.0  133000.0                       8.0   
1   27248.0         0.0   27248.0                       8.0   
2   -5000.0         0.0   -5000.0                       0.0   
3   13852.0         0.0   13852.0                       0.0   
4       0.0         0.0       0.0                       8.0   

   breakdowns_labor_0_rate  breakdowns_material_0_rate   Slug     Total  \
0                      0.0                         0.0  193.0  133000.0   
1                      0.0                         0.0  248.0   27248.0   
2                      0.0                         0.0  251.0       0.0   
3                      0.0                         0.0  252.0       0.0   
4                      0.0                         0.0  253.0       0.0   

   Labor0Breakdown_quantity0total0  Labor0Breakdown_hourstotal  \
0                              0.0                         8.0   
1                              0.0                         8.0   
2                              0.0                         0.0   
3                              0.0                         0.0   
4                              0.0                         8.0   

   Labor0Breakdown_ratetotal  target  
0                      165.0       0  
1                        0.0       0  
2                        0.0       0  
3                        0.0       0  
4                        0.0       0

In [15]:
#exports incorrectly filled out change order data 
potential_fraud.to_csv("../potential fraud/incorrect_data.csv")

In [16]:
# identify values outside of the mean that have actual labor rates 
potential_fraud_values=potential_fraud.loc[potential_fraud['breakdowns_labor_0_rate']!=0]

In [17]:
#potential fraudulent rates 
potential_fraud_values.head(40)

total  paid_total  balance  breakdowns_labor_0_hours  \
154  28320.0     23000.0   5320.0                       1.0   
157   2800.0      2800.0      0.0                       1.0   
158   8550.0      8550.0      0.0                       1.0   
161  12320.0     12320.0      0.0                       1.0   
162   1394.0      1394.0      0.0                       1.0   
163  11000.0     11000.0      0.0                       1.0   
164  15440.0      3250.0  12190.0                       1.0   
165   3675.0      3675.0      0.0                       1.0   
169   2648.0         0.0   2648.0                       1.0   

     breakdowns_labor_0_rate  breakdowns_material_0_rate   Slug   Total  \
154                  28320.0                         0.0  296.0     0.0   
157                   2800.0                         0.0  299.0     0.0   
158                   8550.0                         0.0  301.0     0.0   
161                  12320.0                         0.0  313.0     0.0   
162                   2250.0                         0.0  314.0  1394.0   
163                  11000.0                         0.0  315.0     0.0   
164                  15440.0                         0.0  316.0     0.0   
165                   3675.0                         0.0  317.0     0.0   
169                  11748.0                         0.0  364.0  2648.0   

     Labor0Breakdown_quantity0total0  Labor0Breakdown_hourstotal  \
154                              1.0                         1.0   
157                              1.0                         1.0   
158                              1.0                         1.0   
161                              1.0                         1.0   
162                              1.0                         1.0   
163                              1.0                         1.0   
164                              1.0                         1.0   
165                              1.0                         1.0   
169                              1.0                         1.0   

     Labor0Breakdown_ratetotal  target  
154                    28320.0       0  
157                     2800.0       0  
158                     8550.0       0  
161                    12320.0       0  
162                     2250.0       0  
163                    11000.0       0  
164                      165.0       0  
165                      165.0       0  
169                    11748.0       0

In [18]:
#export potential fraudulent rates
potential_fraud_values.to_csv("../potential fraud/PotentialFraud.csv")